In [ ]:
################################## LENDO A BASE OPERACIONAL DA SEMANA PARA HIGIENIZAR
import pandas as pd
import numpy as np
import pysftp
import io
import math

# CARREGANDO APENAS AS COLUNAS NECESSARIAS PRA HIGIENIZAR (CPF,NOME,TEL,EMAIL)
colunas_necessarias_para_higienizar = ['CPF','NOME_x','TELEFONE','EMAIL','TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO']

telecob_hugo_vigente = pd.read_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO/BASE_OPERACIONAL_TELECOB_HUGO.txt",sep=';', usecols= colunas_necessarias_para_higienizar,dtype=str)


### colocando a base vertical e concatenando
df_1_contratante = telecob_hugo_vigente.loc[:,['CPF','NOME_x','TELEFONE','EMAIL']]
df_2_higienizados = telecob_hugo_vigente.loc[:,['CPF','NOME_x','TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO']]
df_2_higienizados = df_2_higienizados.rename(columns={'TELEFONE_HIGIENIZADO':'TELEFONE'})
df_2_higienizados = df_2_higienizados.rename(columns={'EMAIL_HIGIENIZADO':'EMAIL'})

telecob_hugo_vigente = pd.concat([df_1_contratante,df_2_higienizados])
telecob_hugo_vigente = telecob_hugo_vigente.drop_duplicates(['CPF', 'NOME_x','TELEFONE', 'EMAIL'])

########### ORGANIZANDO OS CPF'S E RESETANDO O INDEX PRA BAIXAR OS ARQUIVOS
telecob_hugo_vigente = telecob_hugo_vigente.sort_values(['CPF'])
telecob_hugo_vigente = telecob_hugo_vigente.reset_index(drop=True)
telecob_hugo_vigente



In [ ]:
# estabelecer conexão SFTP
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None   # ignore host key verification
sftp = pysftp.Connection(host='sftp-afinder.koiketec.com', username='user1009', password='8GEeNgVW5DgbmYyp',port=17592, cnopts=cnopts)


# Definir o tamanho desejado para cada lote

tamanho_lote = math.ceil(len(telecob_hugo_vigente) / 3)

# Calcular o número de lotes necessário
num_lotes = int(np.ceil(len(telecob_hugo_vigente) / tamanho_lote))

# Dividir o DataFrame em lotes
lotes = np.array_split(telecob_hugo_vigente, num_lotes)

# Verificar o tamanho de cada lote e ajustar se necessário
for i, lote in enumerate(lotes):
    if len(lote) > tamanho_lote:
        # Dividir o lote em sublotes menores
        sublotes = np.array_split(lote, len(lote) // tamanho_lote + 1)
        # Substituir o lote original pelos sublotes
        lotes[i:i+1] = sublotes

# Enviar os lotes via SFTP
for i, lote in enumerate(lotes):
    filename = f'NET_lote_{i}_COBRANCA.csv'
    with io.StringIO() as sftp_file:
        lote.to_csv(sftp_file, sep=';', index=False)
        sftp.putfo(io.BytesIO(sftp_file.getvalue().encode()), f'/ftp/Entrada/{filename}')

# Fechar a conexão com o servidor
sftp.close()

In [ ]:
########### ORGANIZANDO OS CPF'S E RESETANDO O INDEX PRA BAIXAR OS ARQUIVOS
telecob_hugo_vigente = telecob_hugo_vigente.sort_values(['CPF'])
telecob_hugo_vigente = telecob_hugo_vigente.reset_index(drop=True)

x = 0
y = 3500000

for qtd in range(3):
    print(f'np_final = df_np.iloc[{x}:{y}]')
    
    np_final = telecob_hugo_vigente.iloc[x:y]
    np_final.to_csv(fr'R:\TI\TELEFONIA\Mailings Prontos\2023\Maio\03\Jorgin da Clock\HIGIENIZACAO CLARO_NET_GEVENUE SEMANA 0305\HIGI_CLARO_NET_GEV_{qtd}_COBRANCA.csv',encoding='utf-8', index=False,sep=';')
    x= y+1
    y= y+3500000
    print(fr'salvando arquivo numero {qtd}')


In [ ]:
######################### RODAR SOMENTE QUANDO FOR PARA VALIDAR AS BASES

In [ ]:
################################## LENDO A BASE OPERACIONAL DA SEMANA PARA VALIDAR
import pandas as pd
import numpy as np
# CARREGANDO APENAS AS COLUNAS NECESSARIAS PRA HIGIENIZAR (CPF,NOME,TEL,EMAIL)
colunas_necessarias_para_validar = ['CPF','NOME_x','TELEFONE','BASE_CONTRATANTE_VALIDADOS','TELEFONE_HIGIENIZADO','BASE_HIGI_VALIDADOS']

telecob_hugo_vigente = pd.read_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO/BASE_OPERACIONAL_TELECOB_HUGO.txt",sep=';'
                                   ,usecols=colunas_necessarias_para_validar,dtype=str)


# ######## FILTRANDO APENAS OS TELEFONES QUE NÃO FORAM VALIDADOS
# telecob_hugo_vigente = telecob_hugo_vigente.query("BASE_CONTRATANTE_VALIDADOS == 'Nao_Validado' and BASE_HIGI_VALIDADOS == 'Nao_Validado' ")
df_1_contratante = telecob_hugo_vigente.query("BASE_CONTRATANTE_VALIDADOS == 'Nao_Validado'")
df_2_higienizados = telecob_hugo_vigente.query("BASE_HIGI_VALIDADOS == 'Nao_Validado' ")

### colocando a base vertical e concatenando
df_1_contratante = df_1_contratante.loc[:,['CPF','NOME_x','TELEFONE']]
df_2_higienizados = df_2_higienizados.loc[:,['CPF','NOME_x','TELEFONE_HIGIENIZADO']]
df_2_higienizados = df_2_higienizados.rename(columns={'TELEFONE_HIGIENIZADO':'TELEFONE'})

telecob_hugo_vigente = pd.concat([df_1_contratante,df_2_higienizados])
telecob_hugo_vigente = telecob_hugo_vigente.drop_duplicates(['CPF', 'NOME_x','TELEFONE'])

# removendo todos os NANS
telecob_hugo_vigente = telecob_hugo_vigente.query("TELEFONE.notnull()")

# FILTRANDO APENAS OS TELEFONES CELULAR '11 DIGITOS'
telecob_hugo_vigente = telecob_hugo_vigente[telecob_hugo_vigente['TELEFONE'].str.len()==11]

## FILTRANDO SOMENTE A COLUNA TELEFONE
telecob_hugo_vigente = telecob_hugo_vigente.loc[:,['TELEFONE']]

In [ ]:
x = 0
y = 390000

for qtd in range(12):
    print(f'np_final = df_np.iloc[{x}:{y}]')
    
    np_final = telecob_hugo_vigente.iloc[x:y]
    np_final.to_csv(fr'R:\TI\TELEFONIA\Mailings Prontos\2023\Maio\03\Jorgin da Clock\VALIDACAO CLARO NET GEV 0305\CLARO_NET_GEV_{qtd}.txt',encoding='utf-8',header=False, index=False,sep=';')
    x= y+1
    y= y+390000
    print(fr'salvando arquivo numero {qtd}')